In [0]:
from pyspark.sql.functions import *

## **Parameters**

In [0]:
catalog = "workspace"
source_schema = "silver"
source_object = "silver_bookings"
cdc_column = "modifiedDate"
backdated_refresh = ""
fact_table = f"{catalog}.{source_schema}.{source_object}"
target_schema = "gold"
target_object = "FactBookings"
fact_key_cols = ["DimCustomersKey", "DimFlightsKey", "DimAirportsKey", "booking_date"]

In [0]:
dimensions = [
    {
        "table" : f"{catalog}.{target_schema}.DimCustomers",
        "alias" : "DimCustomers",
        "join_keys" : [("passenger_id", "passenger_id")]
    },
    {
        "table" : f"{catalog}.{target_schema}.DimFlights",
        "alias" : "DimFlights",
        "join_keys" : [("flight_id", "flight_id")]
    },
    {
        "table" : f"{catalog}.{target_schema}.DimAirports",
        "alias" : "DimAirport",
        "join_keys" : [("airport_id", "airport_id")]
    }
]

# Columns you want to keep from Fact table (besides the surrogate keys)
fact_cols = ["amount", "booking_date", "modifiedDate"]

## **Last Load Date**

In [0]:
if len(backdated_refresh) == 0:
  if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    last_load = spark.sql(f"SELECT MAX({cdc_column}) FROM {catalog}.{target_schema}.{target_object}").collect()[0][0]

  else:
    last_load = "1900-01-01 00:00:00"
else:
  last_load = backdated_refresh

last_load

'1900-01-01 00:00:00'

## **Dynamic Fact Query**


In [0]:
fact_alias = "f"

    # Base columns to select
select_cols = [f"{fact_alias}.{col}" for col in fact_cols]

    #Build joins dynamically
join_clauses = []
for dim in dimensions:
    table_full = dim["table"]
    alias = dim["alias"]
    table_name = table_full.split(".")[-1]
    surrogate_key = f"{alias}.{table_name}Key"
    select_cols.append(surrogate_key)

    on_conditions = [
        f"{fact_alias}.{fk} = {alias}.{dk}" for fk, dk in dim["join_keys"]
    ]
    join_clause = f"LEFT JOIN {table_full} AS {alias} ON" " AND ".join(on_conditions)
    join_clauses.append(join_clause)

# Final SELECT and JOIN clauses
select_clause = ",\n    ".join(select_cols)
joins = "\n".join(join_clauses)

# WHERE clauses for incremenetal filtering
where_clause = f"WHERE {fact_alias}.{cdc_column} >= DATE('{last_load}')"

# Final Query
    query = f"""
SELECT
    {select_clause}
FROM
    {fact_table} AS {fact_alias}
    {joins}
WHERE
    {where_clause}
    """.strip() 

    return query


'f.passenger_id = DimCustomers.passenger_id\nf.flight_id = DimFlights.flight_id\nf.airport_id = DimAirport.airport_id'

In [0]:
def generate_fact_query_incremental(fact_table, dimensions, fact_cols, cdc_column, processing_date):
    fact_alias = "f"

    # Base columns to select
    select_cols = [f"{fact_alias}.{col}" for col in fact_cols]

    #Build joins dynamically
    join_clauses = []
    for dim in dimensions:
        table_full = dim["table"]
        alias = dim["alias"]
        table_name = table_full.split(".")[-1]
        surrogate_key = f"{alias}.{table_name}Key"
        select_cols.append(surrogate_key)

        #Build ON clause
        on_conditions = [
        f"{fact_alias}.{fk} = {alias}.{dk}" for fk, dk in dim["join_keys"]
        ]
        join_clause = f"LEFT JOIN {table_full} {alias} ON " + " AND ".join(on_conditions)
        join_clauses.append(join_clause)

    # Final SELECT and JOIN clauses
    select_clause = ",\n    ".join(select_cols)
    joins = "\n".join(join_clauses)

    # WHERE clauses for incremenetal filtering
    where_clause = f"{fact_alias}.{cdc_column} >= DATE('{last_load}')"

    # Final Query
    query = f"""
SELECT
    {select_clause}
FROM
    {fact_table} {fact_alias}
{joins}
WHERE
    {where_clause}
""".strip()

    return query


In [0]:
query = generate_fact_query_incremental(fact_table, dimensions, fact_cols, cdc_column, last_load)

## **UPSERT**

In [0]:
df_fact = spark.sql(query)

In [0]:
fact_key_cols_str = " AND ".join([f"src.{col} = trg.{col}" for col in fact_key_cols])
fact_key_cols_str

'src.DimCustomersKey = trg.DimCustomersKey AND src.DimFlightsKey = trg.DimFlightsKey AND src.DimAirportsKey = trg.DimAirportsKey AND src.booking_date = trg.booking_date'

In [0]:
from delta.tables import DeltaTable

if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    dlt_obj = DeltaTable.forName(spark, f"{catalog}.{target_schema}.{target_object}")
    dlt_obj.alias("trg").merge(df_fact.alias("src"), fact_key_cols_str)\
                        .whenMatchedUpdateAll(condition = f"src.{cdc_column} >= trg.{cdc_column}")\
                        .whenNotMatchedInsertAll()\
                        .execute()


else:
    df_fact.write.format("delta")\
            .mode("append")\
            .saveAsTable(f"{catalog}.{target_schema}.{target_object}")